In [1]:
import numpy as np
import sys
sys.path.append('/home/yungdankblast/Desktop/Research/delay-optimizer')
from DelayEffects.Analyzer import Analyzer
from DelayEffects.Analyzer_new import FuncOpt
from Optimizer_Scripts.DelayTypeGenerators import *
import os
from matplotlib import pyplot as plt
import matplotlib.animation as animation
import matplotlib as mpl
import pandas as pd

In [2]:
# Initialize
dimensions = [2, 10, 100, 1000]
ack = {d:FuncOpt('Ackley', d) for d in dimensions}
rast = {d:FuncOpt('Rastrigin', d) for d in dimensions}
rosen = {d:FuncOpt('Rosenbrock', d) for d in dimensions}
zak = {d:FuncOpt('Zakharov', d) for d in dimensions}

In [3]:
# Generate points
num_points = 5
ack[2].initialize_points('random', num_points)
ack[10].initialize_points('random', num_points)
ack[100].initialize_points('random', num_points)
ack[1000].initialize_points('random', num_points)

ack[10].x_inits[:,:2] = ack[2].x_inits
ack[100].x_inits[:,:2] = ack[2].x_inits
ack[1000].x_inits[:,:2] = ack[2].x_inits

In [5]:
rast[2].initialize_points('random', num_points)
rast[10].initialize_points('random', num_points)
rast[100].initialize_points('random', num_points)
rast[1000].initialize_points('random', num_points)

rast[10].x_inits[:,:2] = rast[2].x_inits
rast[100].x_inits[:,:2] = rast[2].x_inits
rast[1000].x_inits[:,:2] = rast[2].x_inits

In [6]:
rosen[2].initialize_points('random', num_points)
rosen[10].initialize_points('random', num_points)
rosen[100].initialize_points('random', num_points)
rosen[1000].initialize_points('random', num_points)

rosen[10].x_inits[:,:2] = rosen[2].x_inits
rosen[100].x_inits[:,:2] = rosen[2].x_inits
rosen[1000].x_inits[:,:2] = rosen[2].x_inits

In [7]:
zak[2].initialize_points('random', num_points)
zak[10].initialize_points('random', num_points)
zak[100].initialize_points('random', num_points)
zak[1000].initialize_points('random', num_points)

zak[10].x_inits[:,:2] = zak[2].x_inits
zak[100].x_inits[:,:2] = zak[2].x_inits
zak[1000].x_inits[:,:2] = zak[2].x_inits

In [6]:
# Compute values in each dimension
ack[2].optimize(Undelayed(), break_opt=True, const_lr=False)#, param_file='test_params.csv')
#ack[10].optimize(Undelayed(), break_opt=True)
#ack[100].optimize(Undelayed(), break_opt=True)
#ack[1000].optimize(Undelayed(), break_opt=True)

KeyError: "['learning_rate'] not found in axis"

In [8]:
# Compute values in each dimension
rast[2].optimize(Undelayed(), break_opt=True)
rast[10].optimize(Undelayed(), break_opt=True)
rast[100].optimize(Undelayed(), break_opt=True)
rast[1000].optimize(Undelayed(), break_opt=True)

In [9]:
# Compute values in each dimension
rosen[2].optimize(Undelayed(), break_opt=True)
rosen[10].optimize(Undelayed(), break_opt=True)
rosen[100].optimize(Undelayed(), break_opt=True)
rosen[1000].optimize(Undelayed(), break_opt=True)

In [10]:
# Compute values in each dimension
zak[2].optimize(Undelayed(), break_opt=True)
zak[10].optimize(Undelayed(), break_opt=True)
zak[100].optimize(Undelayed(), break_opt=True)
zak[1000].optimize(Undelayed(), break_opt=True)

In [14]:
ack[2].optimize(Undelayed(), param_type='undelayed')
print(ack[2].lr_params)

{'max_learning_rate': 1.575, 'min_learning_rate': 0.5192, 'step_size': 100}


In [25]:
ack[1000].get_params(Stochastic(1, 1000), param_type='undelayed')

{'max_learning_rate': 1.7406, 'min_learning_rate': 0.0702, 'step_size': 200}